In [1]:
# WANT: Proj SOG

# Model:

# - SOG/GP TY
# - SOG/GP L3-L5
# - Opp Team Allowed SOG/GP TY
# - Opp Team Allowed SOG/GP L3-L5

# Additional:

# - Any Starting Opp D out
# - Starting Opp G out
# - Back to back games
# - Home vs Away
# - Penalties

homeTeam = "EDM"
awayTeam = "BOS"


In [2]:
# GET TEAM DATA FOR ROSTER PROJ SOG

import pandas as pd

teamUrl = "https://www.hockey-reference.com/teams/" + homeTeam + "/"
fullPageData = pd.read_html(teamUrl)
homeTeamData = fullPageData[1]
homeRosterData = fullPageData[4]
homeRosterData.columns = homeRosterData.columns.droplevel()

# Average shots per game
homeAvgS = homeTeamData['S'].iloc[0] / homeTeamData['GP'].iloc[0]

homeSA = homeTeamData['SA'].iloc[0] / homeTeamData['GP'].iloc[0]
print(f"Opp SA avg: {homeSA}")

SGPData = homeRosterData.loc[:,["Player", "GP","S"]]
SGPData['S/GP'] = SGPData['S']/SGPData['GP']
SGPData.drop(SGPData.tail(1).index,inplace=True)
SGPData['S/GP'] = pd.to_numeric(SGPData['S/GP'])
SGPData = SGPData.loc[SGPData['S/GP'] >= 1.5]
SGPData['H/A'] = 'H'
SGPData

Opp SA avg: 31.753846153846155


,Player,GP,S,S/GP,H/A
0,Connor McDavid,65,288,4.430769,H
1,Leon Draisaitl,63,191,3.031746,H
2,Ryan Nugent-Hopkins,65,157,2.415385,H
3,Zach Hyman,64,236,3.687500,H
4,Tyson Barrie,61,115,1.885246,H
5,Darnell Nurse,65,158,2.430769,H
6,Evan Bouchard,65,130,2.000000,H
8,Warren Foegele,50,93,1.860000,H
9,Evander Kane,24,88,3.666667,H
13,Kailer Yamamoto,41,68,1.658537,H


In [3]:
# GET OPPONENT DATA
# INCLUDES:
#  -  OPPONENTS SHOTS ALLOWED AVERAGE
#  -  LEAGUE SHOTS ALLOWED AVERAGE

awayUrl = "https://www.hockey-reference.com/teams/" + awayTeam + "/"
awayFullPageData = pd.read_html(awayUrl)
awayTeamData = awayFullPageData[1]
awayRosterData = awayFullPageData[4]
awayRosterData.columns = awayRosterData.columns.droplevel()

# Average shots per game
awayAvgS = awayTeamData['S'].iloc[0] / awayTeamData['GP'].iloc[0]

awaySA = awayTeamData['SA'].iloc[0] / awayTeamData['GP'].iloc[0]
leagueAvgSA = awayTeamData['SA'].iloc[1] / awayTeamData['GP'].iloc[1]
print(f"Opp SA avg: {awaySA}")
print(f"League SA avg: {leagueAvgSA}")

awayData = awayRosterData.loc[:,["Player", "GP","S"]]
awayData['S/GP'] = awayData['S']/awayData['GP']
awayData.drop(awayData.tail(1).index,inplace=True)
awayData['S/GP'] = pd.to_numeric(awayData['S/GP'])
awayData = awayData.loc[awayData['S/GP'] >= 1.5]
awayData['H/A'] = 'A'
SGPData = pd.concat([SGPData, awayData], axis=0)
SGPData

Opp SA avg: 29.661290322580644
League SA avg: 31.328125


,Player,GP,S,S/GP,H/A
0,Connor McDavid,65,288,4.430769,H
1,Leon Draisaitl,63,191,3.031746,H
2,Ryan Nugent-Hopkins,65,157,2.415385,H
3,Zach Hyman,64,236,3.687500,H
4,Tyson Barrie,61,115,1.885246,H
5,Darnell Nurse,65,158,2.430769,H
6,Evan Bouchard,65,130,2.000000,H
8,Warren Foegele,50,93,1.860000,H
9,Evander Kane,24,88,3.666667,H
13,Kailer Yamamoto,41,68,1.658537,H


In [7]:
# GET AVG SOG OF LAST 5 GAME
# GET PERCENT OF SOG BASED ON TEAM TOTAL AVG SOG
# GET PERCENT OF SOG BASED ON TEAM TOTAL AVG SOG AND LAST 5 GAME AVG SOG

import numpy as np

playerL5AVG = []
playerTeamSPerc = []
playerTeamSPercL5 = []
for index, row in SGPData.iterrows():
    player = row['Player']
    sGP = row['S/GP']
    homeAway = row['H/A']
    if (homeAway == 'H'):
        playerTeamSPerc.append((sGP/homeAvgS)*100)
    else:
        playerTeamSPerc.append((sGP/awayAvgS)*100)
    print(player)
    url = 'https://www.hockey-reference.com/players/'
    nameSplit = player.split()
    url += nameSplit[1][0]
    url += '/'
    url += nameSplit[1][:5]
    url += nameSplit[0][:2]
    url += '01.html'
    url = url.lower()
    print(url)
    try:
        playerL5 = pd.read_html(url, encoding="iso-8859-1")
        playerL5 = playerL5[0]
        playerL5.columns = playerL5.columns.droplevel()
        playerL5.drop(playerL5.tail(1).index,inplace=True)
        playerL5 = pd.to_numeric(playerL5.loc[:,"S"])
        if (playerL5.mean() > 10):
            playerL5AVG.append(np.nan)
            playerTeamSPercL5.append(np.nan)
        else:
            playerL5AVG.append(playerL5.mean())
            if (homeAway == 'H'):
                playerTeamSPercL5.append((playerL5.mean()/homeAvgS)*100)
            else:
                playerTeamSPercL5.append((playerL5.mean()/awayAvgS)*100)
    except:
        playerL5AVG.append(np.nan)
        playerTeamSPercL5.append(np.nan)



SGPData["S/GP L5"] = playerL5AVG
SGPData["S/GP Team %"] = playerTeamSPerc
SGPData["S/GP L5 Team %"] = playerTeamSPercL5
SGPData

Connor McDavid
https://www.hockey-reference.com/players/m/mcdavco01.html
Leon Draisaitl
https://www.hockey-reference.com/players/d/draisle01.html
Ryan Nugent-Hopkins
https://www.hockey-reference.com/players/n/nugenry01.html
Zach Hyman
https://www.hockey-reference.com/players/h/hymanza01.html
Tyson Barrie
https://www.hockey-reference.com/players/b/barrity01.html
Darnell Nurse
https://www.hockey-reference.com/players/n/nurseda01.html
Evan Bouchard
https://www.hockey-reference.com/players/b/bouchev01.html
Warren Foegele
https://www.hockey-reference.com/players/f/foegewa01.html
Evander Kane
https://www.hockey-reference.com/players/k/kaneev01.html
Kailer Yamamoto
https://www.hockey-reference.com/players/y/yamamka01.html
Jesse Puljujarvi
https://www.hockey-reference.com/players/p/puljuje01.html
Mattias Ekholm
https://www.hockey-reference.com/players/e/ekholma01.html
Nick Bjugstad
https://www.hockey-reference.com/players/b/bjugsni01.html
David Pastrnak
https://www.hockey-reference.com/players

,Player,GP,S,S/GP,H/A,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5,Agg Proj S
0,Connor McDavid,65,288,4.430769,H,4.2,13.211009,12.522936,3.918556,3.714464,4.065947
1,Leon Draisaitl,63,191,3.031746,H,2.4,9.039610,7.155963,2.681265,2.122551,2.558890
2,Ryan Nugent-Hopkins,65,157,2.415385,H,2.6,7.201835,7.752294,2.136157,2.299430,2.362743
3,Zach Hyman,64,236,3.687500,H,3.0,10.994839,8.944954,3.261211,2.653189,3.150475
4,Tyson Barrie,61,115,1.885246,H,3.4,5.621146,10.137615,1.667304,3.006947,2.489874
5,Darnell Nurse,65,158,2.430769,H,1.8,7.247706,5.366972,2.149763,1.591913,1.993111
6,Evan Bouchard,65,130,2.000000,H,2.0,5.963303,5.963303,1.768793,1.768793,1.884396
8,Warren Foegele,50,93,1.860000,H,1.2,5.545872,3.577982,1.644977,1.061276,1.441563
9,Evander Kane,24,88,3.666667,H,3.4,10.932722,10.137615,3.242786,3.006947,3.329100
13,Kailer Yamamoto,41,68,1.658537,H,3.4,4.945178,10.137615,1.466804,3.006947,2.383072


TEMPLATE MATH:

Opponents Allowed SOG Avg

Example : 35.23

Avg SOG

Example: 35.4

Toffoli Avg:

~ 8.7% of the SOG for team

8.7% of 35.23 = Proj 3.065 SOG

Toffoli AVG L5:

~ 9.6%

9.6% of 35.23 = 3.382 SOG

AVG OF ALL SOG AVGS:

3.382 + 3.065 + 3.070 + 3.4 = Aggregate project SOG 3.229

In [5]:
# GETS PROJECTED SOG BASED ON AVG PERCENT OF SOG FROM TEAM TOTAL AND ALLOWED SOG FROM OPPONENT
# GETS SAME AS ABOVE, JUST L5 GAMES

projSOG = []
projSOGL5 = []
for index, row in SGPData.iterrows():
    sPerc = row['S/GP Team %']/100
    sPercL5 = row['S/GP L5 Team %']/100
    projSOG.append(sPerc*awaySA)
    projSOGL5.append(sPercL5*awaySA)
    
SGPData["Proj S"] = projSOG
SGPData["Proj S L5"] = projSOGL5
SGPData

,Player,GP,S,S/GP,H/A,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5
0,Connor McDavid,65,288,4.430769,H,4.2,13.211009,12.522936,3.918556,3.714464
1,Leon Draisaitl,63,191,3.031746,H,2.4,9.039610,7.155963,2.681265,2.122551
2,Ryan Nugent-Hopkins,65,157,2.415385,H,2.6,7.201835,7.752294,2.136157,2.299430
3,Zach Hyman,64,236,3.687500,H,3.0,10.994839,8.944954,3.261211,2.653189
4,Tyson Barrie,61,115,1.885246,H,3.4,5.621146,10.137615,1.667304,3.006947
5,Darnell Nurse,65,158,2.430769,H,1.8,7.247706,5.366972,2.149763,1.591913
6,Evan Bouchard,65,130,2.000000,H,2.0,5.963303,5.963303,1.768793,1.768793
8,Warren Foegele,50,93,1.860000,H,1.2,5.545872,3.577982,1.644977,1.061276
9,Evander Kane,24,88,3.666667,H,3.4,10.932722,10.137615,3.242786,3.006947
13,Kailer Yamamoto,41,68,1.658537,H,3.4,4.945178,10.137615,1.466804,3.006947


In [6]:
# GETS AGGREGATE PROJ SOG BASED ON THE STATS BELOW
#  -  AVG SOG
#  -  AVG SOG L5 GAMES
#  -  PROJ SOG BASED ON OPPONENT AND WHOLE SEASON
#  -  PROJ SOG BASED ON OPPONENT AND AVG L5 GAMES

aggProjSOG = []
for index, row in SGPData.iterrows():
    aggProj = row['S/GP'] + row['S/GP L5'] + row['Proj S'] + row['Proj S L5']
    aggProjSOG.append(aggProj/4)
    
SGPData["Agg Proj S"] = aggProjSOG
SGPData

,Player,GP,S,S/GP,H/A,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5,Agg Proj S
0,Connor McDavid,65,288,4.430769,H,4.2,13.211009,12.522936,3.918556,3.714464,4.065947
1,Leon Draisaitl,63,191,3.031746,H,2.4,9.039610,7.155963,2.681265,2.122551,2.558890
2,Ryan Nugent-Hopkins,65,157,2.415385,H,2.6,7.201835,7.752294,2.136157,2.299430,2.362743
3,Zach Hyman,64,236,3.687500,H,3.0,10.994839,8.944954,3.261211,2.653189,3.150475
4,Tyson Barrie,61,115,1.885246,H,3.4,5.621146,10.137615,1.667304,3.006947,2.489874
5,Darnell Nurse,65,158,2.430769,H,1.8,7.247706,5.366972,2.149763,1.591913,1.993111
6,Evan Bouchard,65,130,2.000000,H,2.0,5.963303,5.963303,1.768793,1.768793,1.884396
8,Warren Foegele,50,93,1.860000,H,1.2,5.545872,3.577982,1.644977,1.061276,1.441563
9,Evander Kane,24,88,3.666667,H,3.4,10.932722,10.137615,3.242786,3.006947,3.329100
13,Kailer Yamamoto,41,68,1.658537,H,3.4,4.945178,10.137615,1.466804,3.006947,2.383072
